<a href="https://colab.research.google.com/github/justinmak08/Lab-1-Sim/blob/main/Lab1_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To edit, save a copy of this CoLab notebook using the Save to Drive button above, or Copy to Drive under the Edit menu.

In [ ]:
#@markdown #Simulation Setup { run: "auto" }

# Hey there! Aren't you clever. You shouldn't need to modify any 
# of the code in here, but if you feel confident in your coding 
# abilities you are more than welcome to try.

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.animation import FuncAnimation
from matplotlib import rc
from IPython.display import HTML, display

#@markdown Number of balls:
N       = 5 #@param {type: "slider", min: 2, max: 50}

#@markdown Mass of each ball, in kg:
mass    =   0.5#@param {type: "number"}

#@markdown Spring constant of each connection between balls, in N/m
k_const =  3#@param {type: "number"}

#@markdown Driver frequency, in Hz
drive_freq = 0.20 #@param {type: "number"}

#@markdown Wave pulse?
pulse = True #@param {type: "boolean"}

#@markdown Simulation time step, in seconds:
sim_dt      = 0.01 #@param {type: "number"}

#@markdown Run longer animation (for higher harmonics)? This will take longer.
run_longer = False #@param {type: "boolean"}


# Set time between animations
anim_dt = 1/drive_freq/10
if anim_dt>1:
  anim_dt = np.floor(anim_dt)
elif anim_dt>0.1:
  anim_dt = np.floor(anim_dt*10)/10
elif anim_dt>0.01:
  anim_dt = np.floor(anim_dt*100)/100


# Set total number of animation frames
num_frames = 100
if run_longer:
  num_frames = 400


frames_time = [0]*num_frames
frames_ball_positions = [0]*num_frames
frames_energy = [0]*num_frames



class spring_class:
  def __init__(self, id):
    self.num = id
    self.k = 1



class ball_class:
  def __init__(self, id, spring_left, spring_right):
    self.num = id
    self.spring_left = spring_left
    self.spring_right = spring_right
    self.m = 1
    self.y = 0
    self.v = 0
    self.y_new = 0
    self.v_new = 0


  @property
  def k_left(self):
    if self.spring_left is None:
      print('Trying to get spring constant from spring that does not exist.')
    else:
      return self.spring_left.k


  @property
  def k_right(self):
    if self.spring_right is None:
      print('Trying to get spring constant from spring that does not exist.')
    else:
      return self.spring_right.k




class euler:
  def __init__(self, sim_dt, N):
    self.sim_dt = sim_dt
    self.N = N
    self.t = 0
    self.springs = []
    self.balls = []
    spring_left = None
    for i in range(N):
      if i<N-1:
        spring_right =  spring_class(i)
        self.springs.append(spring_right)
      else:
        spring_right = None
      self.balls.append(ball_class(i, spring_left, spring_right))
      spring_left = spring_right


  def get_ball_nums(self):
    return [ball.num for ball in self.balls]


  def get_ball_masses(self):
    return [ball.m for ball in self.balls]


  def get_spring_nums(self):
    return [spring.num for spring in self.springs]


  def get_spring_constants(self):
    return [spring.k for spring in self.springs]

  # Gets the current y value of all balls
  def get_y_values(self):
    return [ball.y for ball in self.balls]

  # Calculates kinetic energy of all balls
  def kinetic_energy(self):
    ke = 0
    for ball in self.balls:
      ke += 0.5 * ball.m * ball.v**2
    return ke

  # Calculates potential energy in all springs
  def potential_energy(self):
    pe = 0
    for i in range(N-1):
      ball = self.balls[i]
      right = self.balls[i+1]
      pe += 0.5 * ball.k_right * (right.y-ball.y)**2
    return pe

  # Updates time, y positions and velocities to new values
  def update(self):
    for ball in self.balls:
      ball.y = ball.y_new
      ball.v = ball.v_new
    self.t += self.sim_dt

  # Moves simulation forward by a time step of sim_dt
  def advance_sim(self):
    t = self.t
    dt = self.sim_dt
    balls = self.balls
    N = self.N

    # Calculate new position, velocity of left most ball
    leftmost_ball_position(t, dt, balls[0], balls[1])

    # Calculate new position, velocity of left most ball
    rightmost_ball_position(t, dt, balls[N-1], balls[N-2])

    # Calculate new position, velocity of middle balls
    for i in range(1, N-1):
      ball_position(t, dt, balls[i], balls[i-1], balls[i+1])
    
    self.update()


  def advance_to_time(self, time):
    while self.t < time:
      self.advance_sim()



# This makes the loading bar go
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
#@markdown #1) Calculations for the Leftmost Ball
#@markdown The (hidden) code in this cell calculates the position and velocity of the leftmost ball.
#@markdown Double-click this text to reveal the code for editing.

def leftmost_ball_position(t, dt, ball, right):
  # Oscillate sinusoidally
  ball.y_new = np.sin(2.*np.pi*drive_freq*t)

  # Make pulse
  if pulse:
    pulse_duration = 0.5/drive_freq
    if t > pulse_duration:
      ball.y_new = 0

In [ ]:
#@markdown #2) Calculations for the Rightmost Ball
#@markdown The (hidden) code in this cell calculates the position and velocity of the rightmost ball.
#@markdown Double-click this text to reveal the code for editing.

def rightmost_ball_position(t, dt, ball, left):
  # Fixed end
  ball.y_new = 0

In [ ]:
#@markdown #3) Calculations for Ball Position and Velocity
#@markdown The (hidden) code in this cell calculates the position and velocity of any ball that is not the leftmost nor rightmost ball.

#@markdown Double-click this text to reveal the code for editing.

def ball_position(t, dt, ball, left, right):
  # Calculate the force on the ball
  F_left = -1 * ball.k_left * (ball.y - left.y) 
  F_right = -1 * ball.k_right * (ball.y - right.y)
  F_net = F_left + F_right

  # Calculate the acceleration of the ball
  a = F_net/ball.m

  # Calculate the velocity of the ball after a time interval dt
  ball.v_new = ball.v + a*dt

  # Calculate the position of the ball after a time interval dt 
  ball.y_new = ball.y + ball.v_new*dt

In [ ]:
#@markdown #4) Code to Initiate the Simulation and Run the Calculations
#@markdown The (hidden) code in this cell initializes the values from the simulation setup above.
#@markdown Then, the code runs the numerical model.
#@markdown Double-click this text to reveal the code for editing.


####################################################################
############### Code to initialize simulation ######################
####################################################################

sim = euler(sim_dt, N)
balls = sim.balls
springs = sim.springs

for ball in balls:
  ball.m = mass
  #if ball.num > N/2:
  #  ball.m = 4*mass

for spring in springs:
  spring.k = k_const
  #if spring.num > N/2:
  #  spring.k = 4*k_const


# Initalize progress bar
progbar = display(progress(0, num_frames), display_id=True)

for frame in range(num_frames):
  sim.advance_to_time(frame*anim_dt)
  progbar.update(progress(frame, num_frames-1))
  frames_time[frame] = sim.t
  frames_ball_positions[frame] = sim.get_y_values()
  ke = sim.kinetic_energy()
  pe = sim.potential_energy()
  frames_energy[frame] = ke + pe



####################################################################
##################### Code to make plots ###########################
####################################################################

#@markdown ##Plots
#@markdown If the tick-box below is selected, the following plots are also created:
#@markdown * Total Energy (ke+pe) vs. Time
#@markdown * Ball Mass vs. Ball Position along string
#@markdown * Spring Constant vs. Spring Position along string
show_plots = False #@param {type: "boolean"}

ball_nums = sim.get_ball_nums()
ball_masses = sim.get_ball_masses()
spring_nums = sim.get_spring_nums()
spring_constants = sim.get_spring_constants()

if show_plots:
  fig1, (ax11, ax12, ax13) = plt.subplots(1,3, figsize=(12,4))
  fig1.tight_layout()
  ax11.plot(frames_time, frames_energy)
  ax11.set_xlabel("Simulation time (s)")
  ax11.set_ylabel("Energy (J)")
  ax12.plot(ball_nums, ball_masses,'o')
  ax12.set_xlabel("Ball #")
  ax12.set_ylabel("Mass (kg)")
  ax13.plot(spring_nums, spring_constants,'o')
  ax13.set_xlabel("Spring #")
  ax13.set_ylabel("Spring constant (N/m)")
  plt.show()

In [ ]:
#@markdown #5) Code to Animate the Simulation
#@markdown The (hidden) code in this cell animates the simulation.
#@markdown Double-click this text to reveal the code for editing.

# Init progress bar
progbar = display(progress(0, num_frames), display_id=True)


fig = plt.figure(figsize=(8,6))
fig.tight_layout()
ax = plt.axes(xlim=(0, N-1), ylim=(-10, 10))
if pulse:
  ax.set_ylim(-2, 2)
ax.set_title("Waves on a String")
ax.set_xlabel('Ball #')
ax.set_ylabel('Y(t)')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.grid(True)
ax.text(0.3,-0.15,s="Simulation timestep = {} s".format(sim_dt), transform=ax.transAxes)

line, = ax.plot([], [], 'o-', c='r')
text = ax.text(0,-0.15,s="", transform=ax.transAxes)


def animate(frame_num):
  y = frames_ball_positions[frame_num]
  t = frames_time[frame_num]
  line.set_data(ball_nums, y)
  text.set_text("t = {:.4f} s".format(t))
  progbar.update(progress(frame_num, num_frames))

  return line,

anim = FuncAnimation(fig, animate, frames=num_frames, interval=200, blit=True)
#anim = FuncAnimation(fig, animate, init_func=init, frames=num_frames, interval=200, blit=True)
plt.close()

rc('animation', html='jshtml')
anim
